In [2]:
import pandas as pd
import numpy as np

#nilearn imports
from nilearn import plotting, image, interfaces
from nilearn.image import mean_img
from nilearn.plotting import plot_anat, plot_img, plot_stat_map, show, plot_design_matrix
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.reporting import get_clusters_table
import nilearn

#sklearn imports
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [8]:
#function to get first level weights for the runs
#possible task types include colorWheel or sameDifferent 

def extract_beta_weights(subject_id = None, task_type = 'colorwheel',  n_runs=1):
    """
    extract_beta_weights runs first level analysis on each of the subjects, 
    storing their weights for each task type and run in a dictionary.

    subject_id: subject ID 
    task_type: colorWheel or sameDifferent 
    n_runs: max 4 runs, use less when testing
    """
    #parameters
    tr = 2.0 

    # choose appropriate hrf model
    hrf_model = "spm + derivative"
    smoothing_fwhm = 6 # gaussian kernel width (in mm)
    drift_model = None  # cosine drift terms already in confounds
    high_pass=None,  # drift terms equivalent to high-pass filter
    n_jobs=-2,  # use all-1 available CPUs

    # choose whatever confounds you want to include
    interested_confounds = ['white_matter']

    for num_run in range(n_runs):
        preproc_path = f"../teams/a05/group_1_data/fmriprep/sub-{subject_id}/func/sub-{subject_id}_task-{task_type}**run-{n_runs}**.nii.gz"
        events_path = f"../teams/a05/group_1_data/fmriprep/sub-{subject_id}/func/sub-{subject_id}_task-{task_type}**run-{n_runs}_events.tsv"

        #load subject nii files
        run_img = image.load_img(preproc_path)

        #load the event file for the run
        events = pd.read_csv(events_path, sep="\t", 
                            usecols=["onset", "duration"]).assign(
                            trial_type="colorwheel")

        #include confounds
        confounds = interfaces.fmriprep.load_confounds_strategy(
            preproc_path, denoise_strategy="simple")[0][interested_confounds]

        #run the first level model
        fmri_glm = FirstLevelModel(
            t_r=tr,
            hrf_model=hrf_model,
            smoothing_fwhm=smoothing_fwhm,
            drift_model=drift_model,
            minimize_memory=False   
        )

        fmri_glm = fmri_glm.fit(run_img, events, confounds)

        #design matrix = task (convolved with HRF) + confounds
        design_matrix = fmri_glm.design_matrices_[0]

        # map of parameter estimates / beta weights
        # this is the 'feature' map to use in classification
        beta_weights = fmri_glm.compute_contrast("colorwheel", output_type="effect_size")
    return beta_weights

extract_beta_weights(103, 'sameDifferent')['sameDifferent']['run_1']

ValueError: File not found: '../teams/a05/group_1_data/fmriprep/sub-103/func/sub-103_task-sameDifferent**run-1**.nii.gz'

In [ ]:
def train_model(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # SVM classifier
    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy: " + str(accuracy))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    show()

    return svm_model